In [6]:
import pickle
from pathlib import Path

import matplotlib.pyplot as plt
import torch
from neuralhydrology.evaluation import metrics
from neuralhydrology.nh_run import start_run, eval_run

In [7]:
# by default we assume that you have at least one CUDA-capable NVIDIA GPU
if torch.cuda.is_available():
#    start_run(config_file=Path("./config_files/co_basins.yml"))
#    start_run(config_file=Path("./config_files/1_hydroatlas_basin.yml"))
    start_run(config_file=Path("./config_files/30_hydroatlas_basin.yml"))

# fall back to CPU-only mode
else:
#    start_run(config_file=Path("./config_files/co_basins.yml"), gpu=-1)
#    start_run(config_file=Path("./config_files/1_hydroatlas_basin.yml"), gpu=-1)
    start_run(config_file=Path("./config_files/30_hydroatlas_basin.yml"), gpu=-1)

2024-06-11 14:41:41,463: Logging to /Users/joshsturtevant/Documents/lynker/projects/nasa_water/neuralhydrology/2023_colorado_swe/runs/test_run_30_colorado_hydroatlas_basin_1106_144141/output.log initialized.
2024-06-11 14:41:41,465: ### Folder structure created at /Users/joshsturtevant/Documents/lynker/projects/nasa_water/neuralhydrology/2023_colorado_swe/runs/test_run_30_colorado_hydroatlas_basin_1106_144141
2024-06-11 14:41:41,465: ### Run configurations for test_run_30_colorado_hydroatlas_basin
2024-06-11 14:41:41,466: experiment_name: test_run_30_colorado_hydroatlas_basin
2024-06-11 14:41:41,467: train_basin_file: basin_txt_files/30_hydroatlas_basin.txt
2024-06-11 14:41:41,467: validation_basin_file: basin_txt_files/30_hydroatlas_basin.txt
2024-06-11 14:41:41,468: test_basin_file: basin_txt_files/30_hydroatlas_basin.txt
2024-06-11 14:41:41,468: train_start_date: 1980-10-01 00:00:00
2024-06-11 14:41:41,469: train_end_date: 1999-09-30 00:00:00
2024-06-11 14:41:41,469: validation_star

In [8]:
thislinewillstopthenotebookfromrunning

NameError: name 'thislinewillstopthenotebookfromrunning' is not defined

### Evaluate run on test set
The run directory that needs to be specified for evaluation is printed in the output log above. Since the folder name is created dynamically (including the date and time of the start of the run) you will need to change the `run_dir` argument according to your local directory name. By default, it will use the same device as during the training process.

In [5]:
run_dir = Path("runs/test_run_30_colorado_hydroatlas_basin_1106_095251")
#eval_run(run_dir=run_dir, period="test")

eval_run(run_dir=run_dir, period="train")

IndexError: list index out of range

### Load and inspect model predictions
Next, we load the results file and compare the model predictions with observations. The results file is always a pickled dictionary with one key per basin (even for a single basin). The next-lower dictionary level is the temporal resolution of the predictions. In this case, we trained a model only on daily data ('1D'). Within the temporal resolution, the next-lower dictionary level are `xr`(an xarray Dataset that contains observations and predictions), as well as one key for each metric that was specified in the config file.

In [ ]:
with open(run_dir / "test" / "model_epoch050" / "test_results.p", "rb") as fp:
    results = pickle.load(fp)
    
results.keys()

The data variables in the xarray Dataset are named according to the name of the target variables, with suffix `_obs` for the observations and suffix `_sim` for the simulations.

In [ ]:
example_test_basin = '06614800'

In [ ]:
results[example_test_basin]['1D']['xr']

Let's plot the model predictions vs. the observations

In [ ]:
# extract observations and simulations
qobs = results[example_test_basin]['1D']['xr']['QObs(mm/d)_obs']
qsim = results[example_test_basin]['1D']['xr']['QObs(mm/d)_sim']

fig, ax = plt.subplots(figsize=(8,5))
ax.plot(qobs['date'], qobs, label="streamflow observation")
ax.plot(qsim['date'], qsim, label="LSTM prediction")
plt.legend()
ax.set_ylabel("Discharge (mm/d)")
ax.set_title(f"Test period on basin {example_test_basin} - NSE {results[example_test_basin]['1D']['NSE']:.3f}")
plt.show()
plt.close()

Next, we are going to compute all metrics that are implemented in the NeuralHydrology package. You will find additional hydrological signatures implemented in `neuralhydrology.evaluation.signatures`.

In [ ]:
values = metrics.calculate_all_metrics(qobs.isel(time_step=-1), qsim.isel(time_step=-1))
for key, val in values.items():
    print(f"{key}: {val:.3f}")